# Dashboard Interactivo para el Análisis de Meteoritos

Este Jupyter Notebook construye un **dashboard interactivo** (utilizando [Panel](https://panel.holoviz.org/) y otras librerías) para analizar el dataset de meteoritos contenido en `data/Meteorite_Landings.csv`. Permite aplicar filtros dinámicos y visualizar:
- Tendencias temporales
- Distribución de masas (escala log)
- Clasificaciones principales
- Mapa mundial interactivo
- Métricas descriptivas y correlaciones

## Estructura de la solución

- **Sección 1**: Importación de librerías y carga del dataset con las funciones definidas en `src/data_loader.py`.
- **Sección 2**: Creación de widgets (controles) de Panel para filtrar por año, masa, clasificación y tipo (Fell/Found).
- **Sección 3**: Visualizaciones interactivas con Panel + Plotly / hvPlot / Folium.
- **Sección 4**: Cálculo y visualización de métricas (estadísticas descriptivas, correlaciones, métricas espaciales).
- **Sección 5**: Construcción de Tabs y despliegue del dashboard completo.

Asegúrate de tener instaladas las dependencias (Panel, hvPlot, Holoviews, Folium, etc.) para que todas las celdas se ejecuten correctamente.

> **Nota**: El archivo CSV de meteoritos se espera en la ruta `data/Meteorite_Landings.csv`, y los módulos se ubican en `src/data_loader.py`, `src/metrics.py` y `src/visualization.py` respectivamente.


## 1. Carga de librerías y dataset

In [80]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas  # habilita .hvplot en DataFrames
import holoviews as hv
import colorcet as cc
import folium

import sys
import os

# Ruta absoluta a la carpeta "src" (asumiendo que notebooks y src están al mismo nivel)
src_path = os.path.abspath(os.path.join('..', 'src'))

# Añadimos la ruta a sys.path si no está incluida
if src_path not in sys.path:
    sys.path.append(src_path)

# Ahora podemos importar
from data_loader import load_meteorite_data
from metrics import geographic_distribution_metrics


# Extensión de Panel para gráficos (Plotly, Holoviews/Bokeh) en el notebook
pn.extension('plotly', 'holoviews', sizing_mode="stretch_width")

# Cargar el dataset
meteorites = load_meteorite_data(file_path='/home/omega/final-project-topicos-1/data/Meteorite_Landings.csv')
print("Total de registros de meteoritos:", len(meteorites))
meteorites.head(5)

Total de registros de meteoritos: 38401


,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation,unnamed:_10,date
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",NaN,1880-01-01
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",NaN,1951-01-01
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",NaN,1952-01-01
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,"(16.88333, -99.9)",NaN,1976-01-01
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,"(-33.16667, -64.95)",NaN,1902-01-01


### Descripción rápida de columnas numéricas

In [81]:
meteorites[['mass', 'year', 'reclat', 'reclong']].describe()

,mass,year,reclat,reclong
count,3.828200e+04,38401.000000,38401.000000,38401.000000
mean,1.558098e+04,1980.604932,-39.122580,61.074319
std,6.273203e+05,137.762650,46.378511,80.647298
min,0.000000e+00,0.000000,-87.366670,-165.433330
25%,6.700000e+00,1986.000000,-76.714240,0.000000
50%,2.934500e+01,1996.000000,-71.500000,35.666670
75%,1.891750e+02,2002.000000,0.000000,157.166670
max,6.000000e+07,2101.000000,81.166670,354.473330


## 2. Definición de Filtros (Widgets con Panel)

Crearemos los controles de usuario (sliders, multiselect, checkboxes) para filtrar:
- **Año** (rango slider)
- **Masa (g)** (rango slider)
- **Clasificación** (multiselect)
- **Tipo** (Fell/Found) (checkbox group)

Definimos además una función `filter_data(...)` que aplica estos filtros a un `DataFrame` y devuelve el subconjunto resultante.

In [82]:
import panel.widgets as pw

# Rango de años
year_min, year_max = int(meteorites['year'].min()), int(meteorites['year'].max())
year_slider = pw.IntRangeSlider(name='Año', start=year_min, end=year_max, value=(year_min, year_max), step=1)

# Rango de masas
mass_min, mass_max = meteorites['mass'].min(), meteorites['mass'].max()
mass_slider = pn.widgets.RangeSlider(
    name='Masa (g)',
    start=mass_min,
    end=mass_max,
    value=(mass_min, mass_max),
    step=round((mass_max - mass_min) / 100, 2)  # un paso razonable
)


# Clasificaciones (recclass)
classes = sorted(meteorites['recclass'].unique())
class_select = pw.MultiSelect(name='Clasificación', options=classes, value=classes[:], size=6)

# Tipo (Fell / Found)
types = meteorites['fall'].unique().tolist()  # en teoría, ["Fell", "Found"]
type_select = pw.CheckBoxGroup(name='Tipo', options=types, value=types[:])

# Función de filtrado
def filter_data(df, year_range, mass_range, classes_selected, types_selected):
    df_filtered = df[(df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    df_filtered = df_filtered[(df_filtered['mass'] >= mass_range[0]) & (df_filtered['mass'] <= mass_range[1])]
    if classes_selected:
        df_filtered = df_filtered[df_filtered['recclass'].isin(classes_selected)]
    if types_selected:
        df_filtered = df_filtered[df_filtered['fall'].isin(types_selected)]
    return df_filtered

# Probar función de filtro con valores iniciales
meteorites_filtered = filter_data(
    meteorites, year_slider.value, mass_slider.value, class_select.value, type_select.value
)
print("Registros tras filtro inicial (debería ser igual al total):", len(meteorites_filtered))
meteorites_filtered.head(3)

Registros tras filtro inicial (debería ser igual al total): 38282


,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation,unnamed:_10,date
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",NaN,1880-01-01
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",NaN,1951-01-01
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",NaN,1952-01-01


## 3. Visualizaciones Interactivas

Aquí creamos diversas funciones que generan gráficos en función de un DataFrame filtrado. Luego, usando `pn.bind`, las enlazamos a la salida de `filter_data(...)`.

### 3.1 Evolución Temporal (cantidad vs. masa por año)

Usaremos Plotly para generar un gráfico con dos ejes Y:
- Eje principal para la **cantidad de meteoritos** por año
- Eje secundario (derecha) para la **masa total** de meteoritos por año

In [83]:
import plotly.graph_objs as go
from panel import bind

def make_timeseries_plot(df):
    by_year = df.groupby('year').agg(
        meteorite_count=('name', 'count'),
        total_mass=('mass', 'sum')
    ).reset_index()
    if by_year.empty:
        fig = go.Figure()
        fig.update_layout(title="Evolución temporal (sin datos)")
        return fig

    fig = go.Figure()

    # Cantidad de meteoritos (eje y principal)
    fig.add_trace(
        go.Scatter(x=by_year['year'], y=by_year['meteorite_count'], mode='lines+markers',
                   name='Cantidad', marker_color='blue')
    )

    # Masa total (eje y secundario)
    fig.add_trace(
        go.Scatter(x=by_year['year'], y=by_year['total_mass'], mode='lines+markers',
                   name='Masa total (g)', marker_color='red', yaxis='y2')
    )

    # Configurar layout con doble eje Y
    fig.update_layout(
        title="Meteoritos por año vs Masa total por año",
        xaxis_title="Año",
        yaxis=dict(title="Cantidad de meteoritos"),
        yaxis2=dict(title="Masa total (g)", overlaying='y', side='right'),
        legend=dict(x=0.01, y=0.95)
    )
    return fig

# Vincular
timeseries_plot = pn.bind(
    make_timeseries_plot,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

### 3.2 Histograma de la distribución de masas (escala logarítmica)

Utilizamos `hvPlot` para crear un histograma con `logy=True`.

In [ ]:
def make_mass_histogram(df):
    if df.empty or df['mass'].dropna().le(0).all():
        return hv.Curve([]).opts(height=300, width=400, title="Histograma de masas (sin datos)")

    df_mass = df[df['mass'] > 0].copy()

    hist = df_mass.hvplot.hist(
        'mass',
        bins=50,
        title="Distribución de masas de meteoritos (log y)",
        height=300,
        width=400,
    ).opts(logy=True, ylim=(0.1, None))

    return hist


mass_hist_plot = pn.bind(
    make_mass_histogram,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

### 3.3 Diagrama de barras (Top 10 clasificaciones)

Muestra los 10 tipos de meteoritos más frecuentes en el subset filtrado, con hvPlot.

In [85]:
def make_classification_bar(df):
    if df.empty:
        return hv.Curve([]).opts(height=300, width=400, title="Clasificaciones (sin datos)")
    # Contar recclass
    top_classes = df['recclass'].value_counts().nlargest(10)
    class_df = top_classes.reset_index()
    class_df.columns = ['Clasificación', 'Cuenta']
    bars = class_df.hvplot.bar(
        x='Clasificación', y='Cuenta', rot=45,
        height=300, width=400,
        title="Top 10 clasificaciones de meteoritos"
    )
    return bars

class_bar_plot = pn.bind(
    make_classification_bar,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

### 3.4 Mapa global interactivo (Folium)

Se muestra un mapa con Marcadores, usando `CircleMarker` para cada meteorito. Se utiliza `MarkerCluster` para agrupar puntos cercanos.

In [86]:
from folium.plugins import MarkerCluster

def make_map(df):
    m = folium.Map(location=[20, 0], zoom_start=2)  # Mapa centrado en [lat=20, lon=0]
    marker_cluster = MarkerCluster().add_to(m)
    if df.empty:
        folium.Marker(location=[0, 0], popup="No hay meteoritos en este rango").add_to(marker_cluster)
        return m

    for _, row in df.iterrows():
        lat, lon = row.get('reclat'), row.get('reclong')
        if pd.isna(lat) or pd.isna(lon):
            continue
        name = row.get('name', 'Unknown')
        mass = row.get('mass', 'N/A')
        year = row.get('year', 'N/A')
        mtype = row.get('fall', '')  # Fell or Found
        recclass = row.get('recclass', '')
        color = 'red' if mtype == 'Fell' else 'blue'
        popup_text = f"<b>{name}</b><br>Año: {year}<br>Masa: {mass} g<br>Clasificación: {recclass}"

        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=popup_text,
            tooltip=str(recclass)
        ).add_to(marker_cluster)
    return m

meteorite_map = pn.bind(
    make_map,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

## 4. Panel de Métricas

Calculamos y desplegamos:
- Estadísticas descriptivas (media, mediana, std) de la masa.
- Correlaciones entre variables numéricas y heatmap.
- Métricas espaciales.


In [87]:
# 4.1 Estadísticas descriptivas
def descriptive_stats(df):
    if df.empty:
        return pd.DataFrame(columns=["Grupo", "Media (g)", "Mediana (g)", "Desvío Std (g)"])
    
    # Estadísticas globales
    global_stats = df['mass'].agg(['mean', 'median', 'std'])

    # Por tipo Fell/Found
    type_stats = df.groupby('fall')['mass'].agg(['mean', 'median', 'std'])
    type_stats = type_stats.rename(index=str).reset_index()
    type_stats = type_stats.rename(columns={
        'fall': 'Grupo',
        'mean': 'Media (g)',
        'median': 'Mediana (g)',
        'std': 'Desvío Std (g)'
    })

    # Fila global como DataFrame y concatenar
    global_row = {
        'Grupo': 'Global',
        'Media (g)': global_stats['mean'],
        'Mediana (g)': global_stats['median'],
        'Desvío Std (g)': global_stats['std']
    }
    global_df = pd.DataFrame([global_row])

    type_stats = pd.concat([type_stats, global_df], ignore_index=True)

    # Redondear valores numéricos
    for col in ["Media (g)", "Mediana (g)", "Desvío Std (g)"]:
        type_stats[col] = type_stats[col].round(2)

    return type_stats

stats_df = pn.bind(
    descriptive_stats,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

In [88]:
# 4.2 Correlaciones numéricas + Heatmap
def correlation_matrix(df):
    numeric_cols = []
    for col in ['mass', 'year', 'reclat', 'reclong']:
        if col in df.columns:
            numeric_cols.append(col)
    if not numeric_cols:
        return pd.DataFrame()  # no numeric data
    corr_matrix = df[numeric_cols].corr().round(2)
    return corr_matrix

corr_df = pn.bind(
    correlation_matrix,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

def make_correlation_heatmap(df):
    corr = correlation_matrix(df)
    if corr.empty:
        return hv.Curve([]).opts(title="Heatmap de correlaciones (no disponible)", height=300, width=400)
    # Convertir a formato largo
    corr_values = corr.stack().reset_index()
    corr_values.columns = ['Var1', 'Var2', 'Correlacion']
    # Heatmap con hvplot
    cmap_use = cc.coolwarm if hasattr(cc, "coolwarm") else "RdBu"
    heatmap = corr_values.hvplot.heatmap(
        x='Var1', y='Var2', C='Correlacion', cmap=cmap_use,
        clim=(-1,1), colorbar=True, height=300, width=400,
        title="Matriz de correlación"
    )
    heatmap = heatmap.opts(xrotation=45)
    return heatmap

corr_heatmap = pn.bind(
    make_correlation_heatmap,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

In [89]:
from data_loader import convert_to_geodataframe
from metrics import geographic_distribution_metrics

# 4.3 Métricas espaciales usando geographic_distribution_metrics
def spatial_metrics_info(df):
    if df.empty:
        return pd.DataFrame(columns=["Métrica", "Valor"])

    try:
        gdf = convert_to_geodataframe(df)
        metrics = geographic_distribution_metrics(gdf)

        # Convertir salida a DataFrame amigable
        if isinstance(metrics, dict):
            return pd.DataFrame(list(metrics.items()), columns=["Métrica", "Valor"])
        else:
            return pd.DataFrame({"Métrica": ["Error"], "Valor": ["Formato no compatible"]})
    except Exception as e:
        return pd.DataFrame({"Métrica": ["Error"], "Valor": [str(e)]})

spatial_df = pn.bind(
    spatial_metrics_info,
    df=pn.bind(
        filter_data,
        meteorites,
        year_slider,
        mass_slider,
        class_select,
        type_select
    )
)

## 5. Construcción del Dashboard (Panel Tabs)

Creamos las pestañas:
- **Visualizaciones**: gráfico temporal, histograma, top clasificaciones, mapa.
- **Métricas**: descriptivas, correlación, heatmap, métricas espaciales.

Arriba colocamos los filtros, de modo que cualquier cambio se refleje en ambas pestañas simultáneamente.

In [90]:
# Panel de visualizaciones
visualizations_panel = pn.Column(
    pn.pane.Markdown("## <span style='font-size:15px; font-weight:bold'>Visualizaciones interactivas</span>"),
    timeseries_plot,
    pn.Row(mass_hist_plot, class_bar_plot),
    pn.pane.HTML("<hr>", height=10),
    pn.pane.Markdown("**Mapa mundial de meteoritos:**"),
    pn.pane.plot.Folium(meteorite_map, height=400)
)

# Panel de métricas
metrics_panel = pn.Column(
    pn.pane.Markdown("### Métricas y Estadísticas"),
    pn.pane.Markdown("**Estadísticas descriptivas (masa en gramos):**"),
    pn.bind(lambda df: pn.pane.DataFrame(df, index=False, sizing_mode='stretch_width', height=120), stats_df),
    pn.pane.Markdown("**Correlación (matriz de Pearson):**"),
    pn.bind(lambda df: pn.pane.DataFrame(df, height=150), corr_df),
    pn.Row(corr_heatmap),
    pn.pane.Markdown("**Métricas espaciales:**"),
    pn.bind(lambda df: pn.pane.DataFrame(df, index=False, sizing_mode='stretch_width', height=150), spatial_df)
)

# Crear Tabs
tabs = pn.Tabs(
    ("📈 Visualizaciones", visualizations_panel),
    ("📊 Métricas", metrics_panel)
)

# Layout final
dashboard = pn.Column(
    pn.pane.Markdown("## Filtros"),
    pn.Row(year_slider, mass_slider, class_select, type_select),
    tabs
)

# Mostrar Dashboard
dashboard


ValueError: Unexpected option 'ymin' for Histogram type across all extensions. Similar options for current extension ('bokeh') are: ['margin'].